In [92]:
from __future__ import division
import os
from os.path import expanduser
import csv
import sys
import re
import numpy as np
import pandas as pd

# Parameters
ExptName 	= 'stim1'
SubjID 		= 0
RunID  		= 1
nTrials = 384
HitRate 	= 18*(1/32) # multiples of 1/32
FARate  	= 3*(1/32) # multiples of 1/32
Hits = HitRate * 32
FAs = FARate * 32
condRects = [(2,0),(2,2),(2,4),(4,0),(4,2),(4,4)]

# Path info
homeDirectory = expanduser("~")
dataDirectory = homeDirectory + os.sep + 'Google Drive/tACS_VWM_ALPHA/data/' + ExptName + os.sep + 's' + str(SubjID) + os.sep + 'setupData/'
directoryFiles = os.listdir(dataDirectory)
fileName = ''

# load setup file
for file in directoryFiles:
    match = re.match('setup-run' + str(RunID) + '.*', file, re.M|re.I)
    if match != None:
        fileName = dataDirectory + match.group()

# Columns -> TriaID, ChangeCondID, WFCond, SubCond, HFCond
setupData = np.genfromtxt(fileName, delimiter=',')
setupData=np.delete(setupData,0,0)
setupData=np.delete(setupData,5,1)

In [93]:
setupData = setupData[setupData[:,2].argsort()]

In [94]:
resps = np.zeros(nTrials)
for i in range(0, 384, 64):
    condHits = Hits
    condFAs = FAs
    for trial in range(i, i+64):
        if setupData[trial][1] != 3 and condHits > 0:
            resps[trial] = 1
            condHits -= 1
        elif setupData[trial][1] == 3 and condFAs > 0:
            resps[trial] = 1
            condFAs -= 1

In [95]:
setupDF = pd.DataFrame(setupData)
respSeries = pd.Series(resps)
conds = setupDF[2]
changeConds = setupDF[1]
trialIDs = setupDF.index
rts = pd.Series(np.zeros(nTrials))
changeTrials = pd.Series(np.zeros(nTrials))
nDistractors = pd.Series(np.zeros(nTrials))
nTargets = pd.Series(np.zeros(nTrials))
for i in range(nTrials):
    if setupDF[1][i] == 3: changeTrials[i] = 0
    else: changeTrials[i] = 1
    nTargets[i] = condRects[int(setupDF[2][i])-1][0]
    nDistractors[i] = condRects[int(setupDF[2][i])-1][1]
outputDF = pd.DataFrame()

In [96]:
outputDF['ChangeTrial'] = changeTrials
outputDF['nDistractors'] = nDistractors
outputDF['nTargets'] = nTargets
outputDF['Cond'] = conds
outputDF['ChangeCond'] = changeConds
outputDF['Response'] = respSeries
outputDF['trialID'] = setupDF.index
outputDF['RT'] = rts
outputDF.to_csv(homeDirectory + os.sep + 'Google Drive/tACS_VWM_ALPHA/data/' + ExptName + os.sep + 's' + str(SubjID) + os.sep + 'runData/run1.csv')

In [97]:

# these ones we can do with setupData array
# thisExp.addData('trialID', VWMTrials[i].trialID)
# thisExp.addData('ChangeTrial', VWMTrials[i].ChangeTrial)
# thisExp.addData('nDistractors', VWMTrials[i].nDistractors)
# thisExp.addData('nTargets', VWMTrials[i].nTargets)
# thisExp.addData('Cond', VWMTrials[i].condNum)
# thisExp.addData('changeHemi', changeTargHemi)
# thisExp.addData('ChangeCond', VWMTrials[i].ChangeCond)
# thisExp.addData('leftTargs', leftTargCount)
# thisExp.addData('rightTargs', rightTargCount)
# thisExp.addData('leftDists', leftDistCount)
# thisExp.addData('rightDists', rightDistCount)

# resp we need to generate with HitRate and False Alarm
# thisExp.addData('Response',resp)

# random
# thisExp.addData('RT', testResponse.rt)

